--- 
Project for the course in Microeconometrics | Summer 2021, M.Sc. Economics, Bonn University | [Philipp Schreiber](https://github.com/pcschreiber1)

# Replication of Henderson, Storeygard, Deichmann (2017) :
## Has climate change driven urbanization in Africa? <a class="tocSkip">   
    
    
---

<div class="alert alert-block alert-info">
    <h5><b>Downloading and viewing this notebook:</b></h5> <p> The original paper, as well as the data and code provided by the authors can be accessed <a href="https://doi.org/10.1016/j.jdeveco.2016.09.001">here</a> </p></div>

## Content:
Extension: 
 - Spatial SUTVA/ <span style="color:orange">**NOTE**:</span> How large are the districts (important for whetehr SUTVA is reasonable or not)
 - Spatially explicit counterfactual framework
     - using exploration of SUTVA and spatial patters
         - spatial autocorrelation: Moran Plot and Moran's I
     - implementing (lagged dependent, lagged 

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker
import statsmodels.formula.api as smf
import seaborn as sns

#For spatial analysis
import geopandas as gpd
import shapely.geometry as geom
import libpysal as lp #For spatial weights

from pysal.viz import splot #exploratory analysis
#from splot.esda import plot_moran #exploratory analysis
from pysal.explore import esda #exploratory analysis
from pysal.model import spreg #For spatial regression

pd.options.display.float_format = "{:,.2f}".format

In [ ]:
from auxiliary.data_import import *
from auxiliary.plots import *

In [1]:
#map_countries()

In [ ]:
# The explanatory variables of interest
a = ["ADsm0_1moistu",
     "ADsm0_2moistu",
     "ADsm0_3moistu",
     "ADsm0_4moistu",
     "extent_agE_ADsm0_2moistu",
     "ADurbfrac",
     "extent_agE",
     "extent_agH",
     "primate"]

---
# 1. Introduction 
---

In this paper Henderson et al. investigate the impact of increasing aridity on urbanization in Sub-Saharan Africa. In particular, the authors address the questions of i) *whether adverse changes in climate induce a push from rural to urban areas* and ii) *how this within-country migration affects incomes in cities/ the nature of the development effect of such an urbanization push on cities*.  They attribute previous studies' lack of significant findings to aggregation due to using national level data and, therefore, use variation at the district and city level. In addition, they construct a new data set of actual census data to overcome the reliance on interpolations, including the use of **night time lights (NTLs)**.

<img src="material/map_countries.PNG" alt="Error: map_countries.png" title="countries in the data set" width="600"/>

The authors outline a simple model of rural-urban migration to frame their analysis of the impact of rising aridity. The model suggests ...
<span style="color:red">***Shorter*** </span> <span style="color:blue">The authors model districs as small open economies which are differentiated by the presence of manufacturing facilities for potentially tradeable goods. In this context, a decline in moisture should only induce local migration in industrialized districts, since in exclusievly rural districts cities are only provide agriculture with services that not traded across districts, the local agricultural sector is not competing for labor.</span> This study employs **fixed effects (FE) regression** to account for all relevant influences. Finally, the authors support this by looking at evidence from India. The authors find strong evidence for the local migration, nut only in districts likely to produce tradable manufacturing goods. Similary, total income of local cities also only increases in districts liekly to produce tradable manufacturing goods. Understanding the patterns and effects of climate change on urbanization  is important both for providing assistance and to guide proactive policy. Africa projected to be especially affected by climate change ... Sucessful spatial and structural adaptation to climate change will critically depend on promoting cities' capacities.



<span style="color:blue">In this notebook, I replicate the results presented in the paper by Lindo et al. (2010). Furthermore, I discuss in detail the identification strategy used by the authors and evaluate the results using multiple robustness checks. My analysis offers general support for the findings of Lindo et al. (2010) and points out some factors which may enable a deeper understanding of the causal relationship explored in the paper. </span> In particular, I show that the SUTVA assumption is violated through spatial interdependence and **introduce a spatially explicity counterfactual model as an alternative**.

<span style="color:blue"> This notebook is structured as follows. In the next section, I present the performance standard model that lays down the theoretical framework for the paper (Section 2). In Section 3, I analyze the identification strategy that Lindo et al. (2010) use to unravel the causal effects of academic probation on student outcomes and Section 4 briefly discusses the empirical strategy the authors use for estimation. Section 5 and Section 6 constitute the core of this notebook. Section 5 shows my replication of the results in the paper and discussion thereof. In Section 6 I conduct various robustness checks and discuss some limitations of the paper. Section 7 offers some concluding remarks. </span>



---
# 2. Theoretical Background/ Literature
---

<span style="color:red">***What should be discussed here?*** </span>
<ul type="circle">  
 <li > <span style="color:blue"> What the literature says about this issues </span> </li>  
 <li> <span style="color:blue">  How urban migration and migration in the context of climate change has been studied </span> </li>  
</ul>

<span style="color:red">***Urban Models*** </span> <span style="color:blue"> 
     Mentioned in Henderson et al. 2017: (Duranton and Puga, 2004), (Desmet and Henderson, 2015)  
     Moreover: One of the earlier papers by henderson on urban model (Black and Henderson, 1999) </span> 

---
# 3. Data
---

In what follows I present the data. A study of 29 countries over the span ?30? years with subnational data require a sophisticated data set. For further detail, refer to the appendix of the original paper.

<h3><u>3.1 Urbanization</u></h3>
(urbanization over 10-15 years as a function of climate chnage over those years)

Most studies employ **population data** from infrequente national census reports and rely heavily on imputations and interpolations. This study includes countries with at least two available censuses with the relevant information for a complete or nearly complete set of sub-national units, where either district boundaries changed little or common units over time can be defined. The authors digitized data from hardycopy census publications. Of the 32 countries, Namibia, Congo-Brazzaville and Liberia were dropped due to issues of district definition or census frequency. The 29 remaining countries count between 2 to 5 censuses between 1960 and 2019. For estimation purposes Kenya is treated as two countries before and after the urban redefinition of the 1990s. While extensive, the sample omits several Sub-Saharan countries, most notably Nigeria, the most populous country on the continent. The countries were omitted either because the censuses did not include the needed information or printed volumes were unavailable. South Africa was excluded because of Post-Apartheid migration. Districs are large ... <span style="color:blue"> Need to explicitly say how "urban population" is measured. </span>

In contrast, high quality **climate data** is much more readily available. To reflect climatic agricultural potential the authors use precipitation over potential evapotranspiration (i.e. system supply over system demand) as a moisture index. The precipitation and temperature data is available from the University of Delaware gridded climate data set while PET is estimated using the Thornthwaite method. The district level climate indicator is then the average grid cell value that overlaps with the corresponding sub-national unit where cells that cross the boundary are weighted.

Sub-national **data on industrialization** is scarce in many census reports - before 1985 this is even true for national level indicators. Henderson et al. draw on the *Oxford Regional Economic Atlas, Africa* (Ady, 1965) which map all industries by type and city location, based on an in-depth analysis from a variety of sources from the late 1950s and early 1960s. From the total 26 industries the authors exclude 10 agricultural processing industries (e.g. brewing, milling) and use the count of different industries as the industrialization indicator. The full 26 industries have been used in robustness checks.

<h3><u>3.2 Income</u></h3>
(impact of annual climate fluctuations on annual city incomes and implied (possibly short-term) migration)

Due to the lack of consistently available **data on total city income**, **night time lights (NTLs)** are used as a proxy. The authors use annual average data from 1992 to 2008 using data from the U.S. Defense Meteorological Satellite Program (DMSP). As is typical in socioeconomic analysis, clouds and rainfally are excluded and only stable light sources are included. City income is then mapped as the sum of the light intensity of every pixel in the outer envelope. For further information on the use of NTLs see ... .  <span style="color:red">***Also*** </span> <span style="color:blue"> Established strong connection NTL and Henderson, Storeygard, and Weil (2012) was one of the first to formally test the relationship between changes in lights and economic growth</span> 

Since traditional **climate indexes** such as the moisture index are not available at such fine resolutions, rainfall is used instead. The data is used from the Rainfall Climatology Version 2, which combines weather station data with satellity information. Each city's hinterland annual average rainfall is then the average of grid-cell values within 30km of the ever-lit area. Since the moisture index split the sample almost evenly, "we calculate average near-city rainfall between 1983 (the first year in the data) and 1991, and split at the sample median".


---
# 4. Identification
--- 

Disaggregated data for such an extended period of time is extremely scarce, hence the analysis can only rely on a few key variables. As the authors explain, the income study can almost be seen as a robustness check of whether migration is really driving the effect here.

(The biggest threat that the authors are concerned about are unobserved confounders.)

(Identification issues with proxies as "collider variables" (SUTVA paper Yale) -> problematic for NTLs?)
Hence, consider the following tree: (showing a fixed effect appropriate tree)

In [ ]:
#import the data
regiondata = get_spatialdata()

In [ ]:
sns.set_theme(style='whitegrid')
sns.histplot(x=regiondata['ADurbfrac'], kde=True)
_ = sns.pairplot(regiondata[a], kind='reg', diag_kind='kde')